In [1]:
from shared.simulators import ConstantEffectSimulator, ModeratedEffectSimulator, DataSimulator, EffectSimulatorCollection
from shared.analyzers import IPWAnalyzer, SIPWAnalyzer, MetaAnalyzer
from shared.ragged_analyzers import RaggedIPWAnalyzer, RaggedSIPWAnalyzer, RaggedMetaAnalyzer
from shared.result_processors import Results
from tqdm import trange
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy

In [2]:
N = 50
T = 100
effect_simulator = ModeratedEffectSimulator("Noisy Moderated Effects", N, 1., 1., 0.3, 1., 0.2, epsilon_sd=1.)
effect_simulator.sample_params()
data_simulator = DataSimulator(N, T).simulate_history()
data_simulator = data_simulator.simulate_a()
data_simulator = data_simulator.simulate_effects(effect_simulator)
ragged = False
if ragged:
    Ts = np.random.choice(np.arange(T//2, T), N)
else:
    Ts = np.array([T]*N)
T_total = Ts.sum().astype(int)
df = pd.DataFrame()
X = np.zeros((T_total, 2))
X_idx_start = 0
for i in range(N):
    T_i = Ts[i]
    X_idx_end = X_idx_start + T_i
    x = data_simulator.x[i, :T_i]
    X_i = np.ones((T_i, 2))
    X_i[:, 1] = x
    X[X_idx_start:X_idx_end] = X_i.copy()
    df_i = pd.DataFrame({
        "id": [i]*T_i,
        "a": data_simulator.a[i, :T_i],
        "p": data_simulator.p[i, :T_i],
        "y": data_simulator.y[i, :T_i]})
    df = pd.concat([df, df_i], axis=0)
    X_idx_start = X_idx_end
df = df.reset_index()

In [3]:
robust = True
dr = True

# ragged_analyzer = RaggedIPWAnalyzer(name="IPW", dr=dr, robust=robust)
ragged_analyzer = RaggedSIPWAnalyzer(name="SIPW", dr=dr, robust=robust)
# ragged_meta_analyzer = ragged_analyzer.fit(df, X)
ragged_meta_analyzer = RaggedMetaAnalyzer(ragged_analyzer, name="Meta")
ragged_meta_analyzer.fit(df, X)

# analyzer = IPWAnalyzer(name="IPW", dr=dr, robust=robust)
analyzer = SIPWAnalyzer(name="SIPW", dr=dr, robust=robust)
# meta_analysis_inferences = analyzer.get_theta_inferences(data_simulator)
meta_analyzer = MetaAnalyzer(analyzer, name="Meta")
meta_analysis_inferences = meta_analyzer.get_theta_inferences(data_simulator)
meta_analysis_inferences.theta_estimates / ragged_meta_analyzer.estimates

array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]])